SQL:

 - Из базы HomeCredit размещенной в файлах \*.csv;
 - Создана локальная база на PostreSQL;
 - Выполнены запросы на локальную базу.


In [1]:
import pandas as pd
import psycopg2
import os

from tqdm import tqdm

import warnings
warnings.simplefilter("ignore")

In [2]:
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DATA_FULL_PATH = "C:\\Temp\\data\\"
DATABASE_NAME = 'home_credit'

In [3]:
DB_ARGS = {
    'database': DATABASE_NAME,
    'host': '127.0.0.1',
    'user': DB_USER,
    'password': DB_PASSWORD
}

Сделаем функции выполнения запросов и возврата результата из БД

In [4]:
def send_sql_query(query: str, args: dict):
    """
    Выполняет запрос к базе.

    :param query: строка с sql запросом.
    :param args: аргументы для подключения в БД.
    """
    conn = psycopg2.connect(**args)
    try:      
        cursor = conn.cursor()
        cursor.execute(query)
        conn.commit()
    except (Exception, psycopg2.Error) as error:
        print("Error while fetching data from PostgreSQL", error)
    finally:
        if conn:
            cursor.close()
            conn.close()

            
def get_df_from_query(query: str, args: dict) -> pd.DataFrame:
    """
    Выполняет запрос к базе.

    :param query: строка с sql запросом.
    :param args: аргументы для подключения в БД.
    
    :return df: датафрейм с результатом.
    """  
    conn = psycopg2.connect(**args)
    df = pd.read_sql(query, conn)
    conn.close()
    return df

#### Объединим test и train

In [5]:
def csv_train_test(train: pd.DataFrame, test: pd.DataFrame):
    if os.path.isfile(train):
        df_train = pd.read_csv(train)
        df_test = pd.read_csv(test)
        result = pd.concat([df_train, df_test], ignore_index=True)
        result.to_csv(DATA_FULL_PATH + "application_train_test.csv",
                      index=False)
        
        os.remove(train)
        os.remove(test)

Удалим данные файлы, т.к мы их уже объединили

#### Создадим список необходимых файлов

In [7]:
from os import listdir
from os.path import isfile, join
list_files = [f for f in listdir(DATA_FULL_PATH) if isfile(join(DATA_FULL_PATH, f))]

In [8]:
def autocreate_table(list_files):
    
    for file in tqdm(list_files):
        
        replacements = {
            'object': 'varchar',
            'float64': 'float',
            'int64': 'int',
            'datetime64': 'timestamp',
            'timedelta64[ns]': 'varchar'
                }
        df = pd.read_csv(DATA_FULL_PATH + file)

        col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(df.columns, df.dtypes.replace(replacements)))
        cols = ", ".join(col for col in df.columns)
        
        sql_schema_query = f"""
            DROP TABLE IF EXISTS {file[:-4]};
            CREATE TABLE {file[:-4]}(
            {col_str})
        """
        
        send_sql_query(sql_schema_query, DB_ARGS)
        
        sql_data_query = f"""
            COPY {file[:-4]}(
              {cols}
            )
            FROM '{DATA_FULL_PATH + file}' DELIMITER ',' CSV HEADER;
            """
        send_sql_query(sql_data_query, DB_ARGS)

In [9]:
autocreate_table(list_files)

Создадим тестовый запрос

In [10]:
sql_query = """
    SELECT * FROM application_train_test LIMIT 1
"""

In [11]:
%%time
get_df_from_query(sql_query, DB_ARGS)

CPU times: total: 15.6 ms
Wall time: 304 ms


,sk_id_curr,target,name_contract_type,code_gender,flag_own_car,flag_own_realty,cnt_children,amt_income_total,amt_credit,amt_annuity,...,flag_document_18,flag_document_19,flag_document_20,flag_document_21,amt_req_credit_bureau_hour,amt_req_credit_bureau_day,amt_req_credit_bureau_week,amt_req_credit_bureau_mon,amt_req_credit_bureau_qrt,amt_req_credit_bureau_year
0,100002,1.0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0


### Упражнения

In [13]:
# вывести средний доход среди всех клиентов

In [14]:
%%time

query = """
SELECT ROUND(AVG(amt_income_total)) AS mean_income
FROM application_train_test
"""
get_df_from_query(query, DB_ARGS)

CPU times: total: 46.9 ms
Wall time: 4.12 s


,mean_income
0,170116.0


In [15]:
# вывести минимальный и максимальный возраст среди всех клиентов

In [16]:
%%time

query = """
SELECT FLOOR(MIN(ABS(days_birth) / 365)) AS min_age,
       FLOOR(MAX(ABS(days_birth) / 365)) AS max_age
FROM application_train_test
"""
get_df_from_query(query, DB_ARGS)

CPU times: total: 15.6 ms
Wall time: 858 ms


,min_age,max_age
0,20.0,69.0


In [17]:
# вывести количество мужчин и женщин

In [18]:
%%time

query = """
SELECT DISTINCT code_gender
FROM application_train_test
"""
get_df_from_query(query, DB_ARGS)

CPU times: total: 15.6 ms
Wall time: 822 ms


,code_gender
0,M
1,F
2,XNA


Загадочный пол обнаружен)

In [19]:
%%time

query = """
SELECT COUNT(*) FILTER (WHERE code_gender = 'M') AS count_men,
       COUNT(*) FILTER (WHERE code_gender = 'F') AS count_women,
       COUNT(*) FILTER (WHERE code_gender != 'F' AND code_gender != 'M') AS count_other
FROM application_train_test
"""
get_df_from_query(query, DB_ARGS)

CPU times: total: 0 ns
Wall time: 859 ms


,count_men,count_women,count_other
0,121125,235126,4


In [20]:
# вывести общую сумму, количество и среднюю сумму, запрошенную клиентами в кредит с авто и без

In [21]:
%%time

query = """
SELECT CASE
           WHEN flag_own_car = 'Y' THEN 'car'
           ELSE 'no_car'
        END AS is_car,
       SUM(amt_credit) AS sum_credit,
       COUNT(amt_credit) AS count_credit,
       ROUND(AVG(amt_credit)) AS mean_credit
       
FROM application_train_test
GROUP BY flag_own_car
"""
get_df_from_query(query, DB_ARGS)

CPU times: total: 15.6 ms
Wall time: 1.22 s


,is_car,sum_credit,count_credit,mean_credit
0,no_car,1.303948e+11,235235,554317.0
1,car,7.900023e+10,121020,652787.0


In [22]:
# вывести доли клиентов с различным образованием

In [23]:
%%time

query = """
WITH count_education_type AS (

    SELECT CAST(COUNT(name_education_type) AS real) AS count_education_type,
           CAST((SELECT COUNT (*) FROM application_train_test) AS real) AS count_clients,
           name_education_type

    FROM application_train_test
    GROUP BY name_education_type)
    
SELECT name_education_type,
       count_education_type / count_clients AS part_of_clients
   
FROM count_education_type
"""
get_df_from_query(query, DB_ARGS)

CPU times: total: 31.2 ms
Wall time: 1.6 s


,name_education_type,part_of_clients
0,Academic degree,0.000575
1,Higher education,0.245271
2,Incomplete higher,0.033687
3,Lower secondary,0.012045
4,Secondary / secondary special,0.708422


In [24]:
# подсчитать количество полных лет для клиентов, у которых есть во владении автомобиль и недвижимость.
# вывести топ 10 по возрастанию

In [25]:
%%time

query = """
SELECT sk_id_curr, 
       (FLOOR(ABS(days_birth) / 365)) AS age_cliens
FROM application_train_test
WHERE flag_own_car = 'Y' AND flag_own_realty = 'Y'
ORDER BY age_cliens
LIMIT 10
"""
get_df_from_query(query, DB_ARGS)

CPU times: total: 15.6 ms
Wall time: 943 ms


,sk_id_curr,age_cliens
0,372716,20.0
1,283094,20.0
2,253377,21.0
3,221068,21.0
4,245666,21.0
5,248727,21.0
6,216327,21.0
7,229697,21.0
8,216772,21.0
9,230835,21.0


In [26]:
# вывести тех клиентов, у кого доход на одного члена семьи в два раза больше, чем в среднем на одного члена семьи по выборке

In [27]:
%%time

query = """
WITH income_one_person AS (
    SELECT sk_id_curr,
           amt_income_total / cnt_fam_members AS income_one_person,
           ROUND(AVG (amt_income_total / cnt_fam_members) OVER ()) AS mean_income_one_person
    FROM application_train_test)

SELECT sk_id_curr
FROM income_one_person
WHERE income_one_person >= 2 * mean_income_one_person
"""
get_df_from_query(query, DB_ARGS)

CPU times: total: 46.9 ms
Wall time: 1.89 s


,sk_id_curr
0,100002
1,100033
2,100053
3,100070
4,100079
...,...
27728,455940
27729,455959
27730,456009
27731,456119


In [28]:
# вывести клиентов старше 60 лет по которым нет данных в bureau

In [29]:
%%time

query = """
WITH cliend_hight_60 AS (
        SELECT sk_id_curr, 
               (FLOOR(ABS(days_birth) / 365)) AS age_cliens
        FROM application_train_test)
        
    
        
SELECT cliend_hight_60.sk_id_curr
FROM cliend_hight_60
LEFT JOIN bureau ON cliend_hight_60.sk_id_curr = bureau.sk_id_curr
WHERE age_cliens > 60 AND bureau.sk_id_bureau IS NULL
"""
get_df_from_query(query, DB_ARGS)

CPU times: total: 15.6 ms
Wall time: 13.4 s


,sk_id_curr
0,101016
1,109918
2,119663
3,123614
4,129220
...,...
5051,125592
5052,230121
5053,232676
5054,297004


In [30]:
# вывести женщин, у которых в истории bureau было больше двух кредитов, просроченных на 61 день и более
# отсортировать в порядке убывания по кол-ву таких кредитов

In [46]:
%%time

query = """
WITH person_overdue AS (
        SELECT app_tt.sk_id_curr AS id_client,
               bureau.credit_day_overdue,
               COUNT(app_tt.sk_id_curr) OVER (PARTITION BY app_tt.sk_id_curr) AS count_credit
        FROM application_train_test AS app_tt
        LEFT JOIN bureau ON app_tt.sk_id_curr = bureau.sk_id_curr
        WHERE code_gender = 'F' and bureau.credit_day_overdue >= 61)

SELECT DISTINCT(id_client),
       count_credit
FROM person_overdue
WHERE count_credit > 2
ORDER BY count_credit DESC
"""
get_df_from_query(query, DB_ARGS)

CPU times: total: 15.6 ms
Wall time: 1.36 s


,id_client,count_credit
0,264144,5
1,142384,4
2,114166,3
3,337741,3
4,374345,3
5,375724,3
6,431820,3
7,436084,3


In [ ]:
# по данным из bureau (БКИ) расчитать долю просрочки в активных займах для каждого клиента
# вывести топ 7 мужчин с наибольшей суммой просрочки, указав для них, помимо прочего,
# сумму активных кредитов и суммы всех кредитов (активных и закрытых).

In [95]:
%%time

'''
part_of_overdue - извлекаем необходимые данные для дальнейшей обработки 
(id, текущую сумму задолженности, сумма кредитов и статус активный кредит или нет)

intermediate_request - избавляемся от деления на 0,
                       рассчитываем долю просрочки в активных займах,
                       создаем отдельный столбец активный кредит,
                       считаем общую сумму кредитов активных и закрытых

Основной запрос - выборка согласно условиям задачи (топ 7 мужчин с наибольшей суммой просрочки)
'''

query = """
WITH part_of_overdue AS (
        SELECT app_tt.sk_id_curr AS sk_id_curr, 
               SUM(bureau.amt_credit_sum_overdue) AS curr_sum_overdue,
               SUM(bureau.amt_credit_sum) AS credit_sum,
               bureau.credit_active AS status_credit
        FROM bureau
        LEFT JOIN application_train_test AS app_tt ON app_tt.sk_id_curr = bureau.sk_id_curr
        WHERE app_tt.code_gender = 'F'
        GROUP BY app_tt.sk_id_curr, bureau.credit_active, app_tt.code_gender),
        
    intermediate_request AS (
        SELECT sk_id_curr,
                CASE
                   WHEN status_credit = 'Active' THEN SUM(curr_sum_overdue) / SUM(credit_sum)
                   ELSE 0
                END AS part_of_overdue,
                SUM(curr_sum_overdue) AS curr_sum_overdue,
                SUM(credit_sum) AS credit_sum,
                CASE
                   WHEN status_credit = 'Active' THEN SUM(credit_sum)
                   ELSE 0
                END AS curr_active_credit,
                status_credit
        FROM part_of_overdue
        WHERE credit_sum > 0
        GROUP BY sk_id_curr, status_credit
        ORDER BY part_of_overdue DESC)
        
SELECT sk_id_curr,
        part_of_overdue,
        curr_sum_overdue,
        curr_active_credit,
        credit_sum
        
FROM intermediate_request
ORDER BY curr_sum_overdue DESC
LIMIT 7
"""
get_df_from_query(query, DB_ARGS)

CPU times: total: 15.6 ms
Wall time: 13.7 s


,sk_id_curr,part_of_overdue,curr_sum_overdue,curr_active_credit,credit_sum
0,355093,0.878756,3756681.000,4275000.0,4275000.0
1,121845,0.923688,2387232.000,2584458.0,2584458.0
2,142384,0.140314,2079509.895,14820399.0,14820399.0
3,264144,0.520968,1958260.500,3758890.5,3758890.5
4,174099,0.680128,1617403.500,2378088.0,2378088.0
5,226707,14.705493,1361214.000,92565.0,92565.0
6,175940,0.616354,1146667.500,1860403.5,1860403.5
